# PL Attribution Starter Pack

**Not** guaranteed to be free of bugs.   Make sure to read and check the code if you use any of it.

In [1]:
import datetime
import logging
import copy

import pandas as pd
import numpy as np
import plotnine as p9


In [2]:
from ragtop.blackscholes import american, black_scholes
from ragtop.implicit import find_present_value, infer_conforming_time_grid, form_present_value_grid, integrate_pde, construct_implicit_grid_structure
from ragtop.instruments import ConvertibleBond, CallableBond
from ragtop.term_structures import spot_to_df_fcn, variance_cumulation_from_vols
from ragtop.extrapolation import spline_extrapolate



In [3]:
logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

logging.getLogger("ragtop").setLevel(logging.WARNING)

## Relevant Data

In [4]:
f_equity = pd.read_csv('data/Ford_Equity_July_2025.csv', index_col="date",  parse_dates=True)
f_equity.reset_index(inplace=True)
f_equity.set_index(
    ['date', 'window_time'], inplace=True
)

f_median_trade_prices = pd.read_csv('data/Ford_Convert_Median_Mkt_Trade_July_2025.csv', index_col="date", parse_dates=True)

f_spreads3y = pd.read_csv('data/Ford_3Y_CDS_July_2025.csv', index_col="date", parse_dates=True)

f_vola = pd.read_csv('data/Ford_Vola_July_2025.csv', index_col="date", parse_dates=True)

yield_curves = pd.read_csv('data/YieldCurves_July_2025.csv', index_col="date", parse_dates=True)
f_spreads3y.iloc[[0,1,-2,-1]].transpose()


date,2025-07-11,2025-07-14,2025-07-24,2025-07-25
shortname,Ford Mtr Co,Ford Mtr Co,Ford Mtr Co,Ford Mtr Co
ticker,F,F,F,F
tier,SNRFOR,SNRFOR,SNRFOR,SNRFOR
runningcoupon,0.05,0.05,0.05,0.05
upfront,-0.099839,-0.099183,-0.098011,-0.097824
tenor,3Y,3Y,3Y,3Y
parspread,0.01363,0.01375,0.013858,0.013884
convspreard,0.013414,0.013534,0.01365,0.013673
cdsrealrecovery,0.4,0.4,0.4,0.4
cdsassumedrecovery,0.4,0.4,0.4,0.4


In [5]:
# Use the fact the data of spread3y is only over our relevant dates
# Some dates had no bond trades, so check those too
dates = [d for d in f_spreads3y.index if d in f_median_trade_prices.index]
', '.join([str(d.date()) for d in dates])

'2025-07-11, 2025-07-14, 2025-07-15, 2025-07-16, 2025-07-17, 2025-07-22, 2025-07-23, 2025-07-24'

In [6]:
f_equity

Unnamed: 0 ticker  best_bid  \
date       window_time                                              
2025-07-11 2025-07-11 10:00:00-04:00           0      F     11.80   
           2025-07-11 11:00:00-04:00           1      F     11.76   
           2025-07-11 12:00:00-04:00           2      F     11.73   
           2025-07-11 13:00:00-04:00           3      F     11.77   
           2025-07-11 14:00:00-04:00           4      F     11.80   
...                                          ...    ...       ...   
2025-07-25 2025-07-25 12:00:00-04:00           2      F     11.28   
           2025-07-25 13:00:00-04:00           3      F     11.32   
           2025-07-25 14:00:00-04:00           4      F     11.38   
           2025-07-25 15:00:00-04:00           5      F     11.41   
           2025-07-25 16:00:00-04:00           6      F     11.46   

                                      best_bidsizeshares  best_ask  \
date       window_time                                               
2025-07-11 2025-07-11 10:00:00-04:00               15800     11.81   
           2025-07-11 11:00:00-04:00               22300     11.77   
           2025-07-11 12:00:00-04:00               28300     11.74   
           2025-07-11 13:00:00-04:00               64200     11.78   
           2025-07-11 14:00:00-04:00               24600     11.81   
...                                                  ...       ...   
2025-07-25 2025-07-25 12:00:00-04:00               54300     11.29   
           2025-07-25 13:00:00-04:00               47700     11.33   
           2025-07-25 14:00:00-04:00               63500     11.39   
           2025-07-25 15:00:00-04:00               60800     11.42   
           2025-07-25 16:00:00-04:00               60300     11.47   

                                      best_asksizeshares  \
date       window_time                                     
2025-07-11 2025-07-11 10:00:00-04:00               11400   
           2025-07-11 11:00:00-04:00               23200   
           2025-07-11 12:00:00-04:00               25300   
           2025-07-11 13:00:00-04:00               30000   
           2025-07-11 14:00:00-04:00               24800   
...                                                  ...   
2025-07-25 2025-07-25 12:00:00-04:00               28000   
           2025-07-25 13:00:00-04:00               36200   
           2025-07-25 14:00:00-04:00               23400   
           2025-07-25 15:00:00-04:00               26600   
           2025-07-25 16:00:00-04:00              156200   

                                                       time_of_last_quote  \
date       window_time                                                      
2025-07-11 2025-07-11 10:00:00-04:00  2025-07-11 09:59:59.467910021-04:00   
           2025-07-11 11:00:00-04:00  2025-07-11 10:59:52.055819985-04:00   
           2025-07-11 12:00:00-04:00  2025-07-11 11:59:59.985827461-04:00   
           2025-07-11 13:00:00-04:00  2025-07-11 12:59:59.669392109-04:00   
           2025-07-11 14:00:00-04:00  2025-07-11 13:59:59.989697437-04:00   
...                                                                   ...   
2025-07-25 2025-07-25 12:00:00-04:00  2025-07-25 11:59:58.593159320-04:00   
           2025-07-25 13:00:00-04:00  2025-07-25 12:59:58.655898256-04:00   
           2025-07-25 14:00:00-04:00  2025-07-25 13:59:59.877015287-04:00   
           2025-07-25 15:00:00-04:00  2025-07-25 14:59:52.715604420-04:00   
           2025-07-25 16:00:00-04:00  2025-07-25 15:59:59.994574217-04:00   

                                      mid_equity_price  
date       window_time                                  
2025-07-11 2025-07-11 10:00:00-04:00            11.805  
           2025-07-11 11:00:00-04:00            11.765  
           2025-07-11 12:00:00-04:00            11.735  
           2025-07-11 13:00:00-04:00            11.775  
           2025-07-11 14:00:00-04:00            11.805  
...                                            

In [7]:
def mkt_data_for_date(d, is_morning=True):
    bond_trade_prices = f_median_trade_prices.loc[d]
    
    yc_in = yield_curves.loc[d]
    mktprice_today_in = bond_trade_prices['today_median']
    mktprice_tomorrow_in = bond_trade_prices['tomorrow_median']
    spread_in = f_spreads3y.loc[d]
    vola_in = f_vola.loc[d]
    equity_in = f_equity.loc[d].iloc[0 if is_morning else 4]
    
    return yc_in, mktprice_today_in, mktprice_tomorrow_in, spread_in, vola_in, equity_in

yc_in, mktprice_today_in, mktprice_tomorrow_in, spread_in, vola_in, equity_in = mkt_data_for_date('2025-07-14')
    


In [8]:
def inputs_for_date(d, is_morning=True, hazard_power=2, proportion_affected=0.5, dvola=0.01, dhazard=0.0002, dr=0.0001):
    if not isinstance(d, pd.Timestamp):
        d = pd.Timestamp(datetime.datetime.strptime(d, '%Y-%m-%d' ).date())
    
    yc_in, mktprice_today_in, mktprice_tomorrow_in, spread_in, vola_in, equity_in = mkt_data_for_date(d, is_morning=is_morning)
    
    # Risk free reates
    yield_curve = pd.DataFrame({'time':[30/360,90/360,180/360],'rate':yc_in.values})
    discounting_f = spot_to_df_fcn(yield_curve)
    
    yc_up   = spot_to_df_fcn(pd.DataFrame({'time':[30/360,90/360,180/360],'rate':(yc_in.values+dr)}))
    yc_down = spot_to_df_fcn(pd.DataFrame({'time':[30/360,90/360,180/360],'rate':(yc_in.values-dr)}))
    
    # Equity price
    S0 = equity_in['mid_equity_price']
    
    # Dividends (from Bloomberg BDVD)
    div_dates = [
        pd.Timestamp(datetime.date(2025,8,11)), 
        pd.Timestamp(datetime.date(2026,11,7)), 
        pd.Timestamp(datetime.date(2026,2,17)), 
        pd.Timestamp(datetime.date(2026,5,11)), 
    ]
    dividends = pd.DataFrame({
        'time': [(exd-d).days/365 for exd in div_dates],
        'fixed': 0.15,
        'proportional': 0,
    })
    
    
    # Credit
    CDS_upfront = spread_in['upfront']
    CDS_parspread = spread_in['parspread']
    CDS_recov = spread_in['cdsassumedrecovery']
    hzd_rate  = CDS_parspread/(1-CDS_recov)
    logger.debug(f"Hazard rate: {hzd_rate:.5f}")
    
    # The ragtop library uses term structures to deal with risk free rates, hazard rrates and volatility
    
    def default_intensity_f(t,S,**kwargs):
        return hzd_rate * ((1-proportion_affected) + proportion_affected * (S0 / S) ** hazard_power)  
    
    def hzf_up_f(t,S,**kwargs):
        return (hzd_rate+dhazard) * ((1-proportion_affected) + proportion_affected * (S0 / S) ** hazard_power)
    
    def hzf_down_f(t,S,**kwargs):
        return (hzd_rate-dhazard) * ((1-proportion_affected) + proportion_affected * (S0 / S) ** hazard_power)
    
    # Vola
    vola_df = pd.DataFrame({
        'time': [int(colname)/360 for colname in vola_in.index],
        'volatility': vola_in.values,
    })
    variance_f = variance_cumulation_from_vols(vola_df)

    vola_df_down = pd.DataFrame({
        'time': vola_df['time'],
        'volatility': vola_df['volatility'].values - dvola,
    })
    variance_down_f = variance_cumulation_from_vols(vola_df_down)
    
    vola_df_up = pd.DataFrame({
        'time': vola_df['time'],
        'volatility': vola_df['volatility'].values + dvola,
    })
    variance_up_f = variance_cumulation_from_vols(vola_df_up)
    
    
    
    
    model_inputs = {
        'Underlying': S0,
        'Avg Trade Price Today': mktprice_today_in,
        'Avg Trade Price Tomorrow': mktprice_tomorrow_in,
        'Default Intensities': default_intensity_f,
        'Recovery Rate': CDS_recov,
        'Risk-free Rates': discounting_f,
        'Vola/Variance': variance_f,
        'Vola/Variance Up': variance_up_f,
        'Vola/Variance Down': variance_down_f,
        'Hazard Up':hzf_up_f,
        'Hazard Down':hzf_down_f,
        'Base Hazard Rate':hzd_rate,
        'Rates Up':yc_up,
        'Rates Down':yc_down,
        'Dividends': dividends,
    }    
    
    return model_inputs
inputs_for_date('2025-07-14')

{'Underlying': np.float64(11.735),
 'Avg Trade Price Today': np.float64(101.4674),
 'Avg Trade Price Tomorrow': np.float64(100.6773),
 'Default Intensities': <function __main__.inputs_for_date.<locals>.default_intensity_f(t, S, **kwargs)>,
 'Recovery Rate': np.float64(0.4),
 'Risk-free Rates': <function ragtop.term_structures.spot_to_df_fcn.<locals>.discount_factor_callable(T: 'float | NDArray[Any]', t: 'float | NDArray[Any]' = 0.0, **_: 'dict[str, Any]') -> 'NDArray[Any]'>,
 'Vola/Variance': <function ragtop.term_structures.variance_cumulation_from_vols.<locals>.var_cum(T: 'float | NDArray[Any]', t: 'float | NDArray[Any]' = 0.0) -> 'NDArray[Any]'>,
 'Vola/Variance Up': <function ragtop.term_structures.variance_cumulation_from_vols.<locals>.var_cum(T: 'float | NDArray[Any]', t: 'float | NDArray[Any]' = 0.0) -> 'NDArray[Any]'>,
 'Vola/Variance Down': <function ragtop.term_structures.variance_cumulation_from_vols.<locals>.var_cum(T: 'float | NDArray[Any]', t: 'float | NDArray[Any]' = 0.0

In [9]:
def compute_attribution_inputs(Tmax, inputs_dict):
    # Attribution parameters:
    # Help us work out day to day -- how much did rates, volatility, hazard change overall on their curves?
    
    r = -np.log(inputs_dict['Risk-free Rates'](Tmax, 0.0)) / Tmax
    sigma0 = np.sqrt(inputs_dict['Vola/Variance'](Tmax)/Tmax)
    S0 = inputs_dict['Underlying']
    h = inputs_dict['Base Hazard Rate']  # not using curves here, at least for now
    # NB: Ignoring borrow cost / divrate
    
    attribution_inputs = {
        'Underlying Price':S0,
        'Avg Risk Free Rate' : r,
        'Avg Volatility' : sigma0,
        'Avg Hazard Rate' : h,
    }
    return attribution_inputs


On each date, we need to run the model not only to get price, but also do finite differences to get sensitivity to rates, default risk, volatility and so on

In [24]:
def run_model_for_date(d, is_morning=True, 
                       hazard_power=2, proportion_affected=0.75, 
                       dividend_rate=0, borrow_cost=0,  # Discrete divs will be from from inputs_for_date()
                       dvola=0.01, dhazard=0.0002,dr=0.0001,
                       n_steps=200, grid_stdevs_width=5):
    
    inputs_dict = inputs_for_date(d, is_morning=is_morning, 
                                  hazard_power=hazard_power, proportion_affected=proportion_affected, 
                                  dvola=dvola, dhazard=dhazard,dr=dr)
    S0 = inputs_dict['Underlying']
    
    maturity = (pd.Timestamp('2026-03-15') - d).days/365
    coups = None
    dconv = ConvertibleBond(
        coupons=coups,
        conversion_ratio=74.5103,
        maturity=maturity,
        calls=None,
        puts=None,
        notional=1000.0,
        recovery_rate=inputs_dict['Recovery Rate'],
        name="FordMarchOf2026",
    )
    
    straightbond = CallableBond(
        coupons=coups,
        maturity=maturity,
        calls=None,
        puts=None,
        notional=1000.0,
        recovery_rate=inputs_dict['Recovery Rate'],
        name="StraightBondEquivalent",
    )
    
    # Need dconv_u, dconv_d etc only for older ragtop without reset_caches() functionality
    dconv_u = copy.deepcopy(dconv)
    dconv_d = copy.deepcopy(dconv)
    dconv_ru = copy.deepcopy(dconv)
    dconv_rd = copy.deepcopy(dconv)
    dconv_hu = copy.deepcopy(dconv)
    dconv_hd = copy.deepcopy(dconv)
    Tmax = dconv.maturity


    K = dconv.notional / dconv.conversion_ratio
    
    attribution_inputs = compute_attribution_inputs(Tmax, inputs_dict)
    r      = attribution_inputs['Avg Risk Free Rate']
    sigma0 = attribution_inputs['Avg Risk Free Rate']
    c = r - dividend_rate - borrow_cost
    
    logger.debug(f"Max T: {Tmax:.2f}. Effective strike {K:.4f}.  Constant-equiv sigma {100*sigma0:.2f}%, r {100*r:.2f}% c {100*c:.2f}%")
    
    def stock_level_fcn(z, t):
        S_levels = K * np.exp(z - (c - 0.5 * sigma0**2) * (Tmax - t))
        return S_levels

    
    grid_structure = construct_implicit_grid_structure(
            tenors=[Tmax],
            M=n_steps,
            S0=S0,
            K=K,
            c=c,
            sigma=sigma0,
            structure_constant=2,
            std_devs_width=grid_stdevs_width,
        )
    time_pts = infer_conforming_time_grid(
            n_steps, Tmax, instruments=[dconv]
        )
    deltaT0 = T1 = time_pts[1]
    
    full_grid = integrate_pde(
            z=grid_structure["z"],
            min_num_time_steps=n_steps,
            S0=S0,
            Tmax=Tmax,
            instruments=[dconv, straightbond],  # Note including straight bond
            stock_level_fcn=stock_level_fcn,
            discount_factor_fcn=inputs_dict['Risk-free Rates'],
            default_intensity_fcn=inputs_dict['Default Intensities'] or (lambda t, S,:  0.0 * S),
            variance_cumulation_fcn=inputs_dict['Vola/Variance'],
            dividends=inputs_dict['Dividends'],
        )
    
    S_T0 = stock_level_fcn(grid_structure["z"], 0.0)
    S_T1 = stock_level_fcn(grid_structure["z"], T1)


    V_T0 = full_grid[0, :, 0]
    V_T1 = full_grid[1, :, 0]
    
    deltaS = S0/100.0

    testS = np.array([S0-deltaS , S0, S0+deltaS])
    
    pv_interpolation = spline_extrapolate(S_T0, V_T0, testS)
    t1_interpolation = spline_extrapolate(S_T1, V_T1, np.array([S0], dtype=float))

    V0 = float(pv_interpolation[1])
    delta = float((pv_interpolation[2] - pv_interpolation[0])/(2*deltaS))
    gamma = float((pv_interpolation[2] - 2*V0 + pv_interpolation[0])/(deltaS**2))
    theta = float(((t1_interpolation[0] - V0)/deltaT0))
    
    straightbond_price = spline_extrapolate(S_T0, full_grid[0, :, 1], np.array([S0]))[0]
    premium = V0 - straightbond_price
    
    
    # Manual finite differencing to get other risk params
    
    v_vola_up = find_present_value(
        S0=S0, 
        instruments=[dconv_u],
        num_time_steps=n_steps,
        default_intensity_fcn=inputs_dict['Default Intensities'],
        discount_factor_fcn=inputs_dict['Risk-free Rates'],
        variance_cumulation_fcn=inputs_dict['Vola/Variance Up'],
        std_devs_width=grid_stdevs_width,
        dividends=inputs_dict['Dividends'],
    )["FordMarchOf2026"]
    v_vola_down = find_present_value(
        S0=S0, 
        instruments=[dconv_d],
        num_time_steps=n_steps,
        default_intensity_fcn=inputs_dict['Default Intensities'],
        discount_factor_fcn=inputs_dict['Risk-free Rates'],
        variance_cumulation_fcn=inputs_dict['Vola/Variance Down'],
        std_devs_width=grid_stdevs_width,
        dividends=inputs_dict['Dividends'],
    )["FordMarchOf2026"]
    vega = (v_vola_up - v_vola_down)/(2*dvola)

    v_hzd_up = find_present_value(
        S0=S0, 
        instruments=[dconv_hu],
        num_time_steps=n_steps,
        default_intensity_fcn=inputs_dict['Hazard Up'],
        discount_factor_fcn=inputs_dict['Risk-free Rates'],
        variance_cumulation_fcn=inputs_dict['Vola/Variance Up'],
        std_devs_width=grid_stdevs_width,
        dividends=inputs_dict['Dividends'],
    )["FordMarchOf2026"]
    v_hzd_down = find_present_value(
        S0=S0, 
        instruments=[dconv_hd],
        num_time_steps=n_steps,
        default_intensity_fcn=inputs_dict['Hazard Down'],
        discount_factor_fcn=inputs_dict['Risk-free Rates'],
        variance_cumulation_fcn=inputs_dict['Vola/Variance Down'],
        std_devs_width=grid_stdevs_width,
        dividends=inputs_dict['Dividends'],
    )["FordMarchOf2026"]
    cdv01 = (v_hzd_up - v_hzd_down)/(2*dhazard)

    v_rates_up = find_present_value(
        S0=S0, 
        instruments=[dconv_ru],
        num_time_steps=n_steps,
        default_intensity_fcn=inputs_dict['Default Intensities'],
        discount_factor_fcn=inputs_dict['Rates Up'],
        variance_cumulation_fcn=inputs_dict['Vola/Variance'],
        std_devs_width=grid_stdevs_width,
        dividends=inputs_dict['Dividends'],
    )["FordMarchOf2026"]
    v_rates_down = find_present_value(
        S0=S0, 
        instruments=[dconv_rd],
        num_time_steps=n_steps,
        default_intensity_fcn=inputs_dict['Default Intensities'],
        discount_factor_fcn=inputs_dict['Rates Down'],
        variance_cumulation_fcn=inputs_dict['Vola/Variance'],
        std_devs_width=grid_stdevs_width,
        dividends=inputs_dict['Dividends'],
    )["FordMarchOf2026"]
    rdv01 = (v_rates_up - v_rates_down)/(2*dr)
    
    risk_params_dict = {'date':d, 'ModelPrice':V0, 'Tenor':Tmax,
                    'Delta':delta, 'Gamma':gamma, 'Theta':theta, 'Vega':vega, 'Rates DV01':rdv01, 'Credit DV01 (drifted)':cdv01,
                    'Convertibility Premium': premium,
                    'Avg Trade Price':inputs_dict['Avg Trade Price Today']*dconv.notional/100,
                   }
    
    overall_dict = risk_params_dict.copy()
    overall_dict.update(attribution_inputs)

    ps = pd.Series(overall_dict)
    
    return ps

run_model_for_date(pd.Timestamp('2025-07-14'), is_morning=True)

date                      2025-07-14 00:00:00
ModelPrice                        1003.552974
Tenor                                0.668493
Delta                               22.976876
Gamma                                2.796595
Theta                              -41.356553
Vega                               265.728932
Rates DV01                        -457.545328
Credit DV01 (drifted)            13043.623714
Convertibility Premium              41.375248
Avg Trade Price                      1014.674
Underlying Price                       11.735
Avg Risk Free Rate                   0.043827
Avg Volatility                       0.309414
Avg Hazard Rate                      0.022916
dtype: object

In [25]:
details_arc_list = [
    run_model_for_date(datadate) for datadate in dates
]
details_arc = pd.DataFrame(details_arc_list).set_index('date')
details_arc.iloc[[0,1,2,-2,-1]].transpose()

date,2025-07-11,2025-07-14,2025-07-15,2025-07-23,2025-07-24
ModelPrice,1006.951769,1003.552974,1004.213764,987.590806,989.341212
Tenor,0.676712,0.668493,0.665753,0.643836,0.641096
Delta,24.461397,22.976876,23.319693,14.958106,15.606761
Gamma,2.799107,2.796595,2.842547,2.075506,2.006987
Theta,-47.368231,-41.356553,-46.136359,-25.496375,-28.885495
Vega,272.162007,265.728932,265.761536,236.065797,238.583624
Rates DV01,-461.117114,-457.545328,-455.356250,-473.915567,-470.093989
Credit DV01 (drifted),13367.742960,13043.623714,13046.972294,11539.623501,11668.476454
Convertibility Premium,45.148876,41.375248,42.020241,24.210378,25.737324
Avg Trade Price,1012.805500,1014.674000,1006.773000,1002.213000,999.381000
